In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Custom Functions
import sys
sys.path.append('../code')
import helper as he
import ner

I0113 15:26:41.951092  3456 file_utils.py:35] PyTorch version 1.3.1 available.
I0113 15:26:45.606345  3456 custom.py:19] [INFO] Project Target Language **en**


In [48]:
nr = ner.NER()

2020-01-13 18:15:32,174 loading file C:/Users/makayser/Desktop/nlp_local//en-ner-ontonotes-fast-v0.4.pt


In [45]:
text = 'I have been using windows 7 and getting some errors with the following code 0x800700c1. What could this mean? I use windows 7 from steve ballmer'

In [49]:
nr.run(text)

[{'value': '7',
  'start': 26,
  'end': 27,
  'label': 'CARDINAL',
  'source': 'flair'},
 {'value': 'ballmer',
  'start': 137,
  'end': 144,
  'label': 'PERSON',
  'source': 'flair'},
 {'value': 'windows 7',
  'start': 4,
  'end': 6,
  'label': 'Product',
  'source': 'list'},
 {'value': 'steve ballmer',
  'start': 26,
  'end': 28,
  'label': 'Boss',
  'source': 'list'},
 {'value': '0x800700c1.',
  'start': 76,
  'end': 87,
  'label': 'ERROR CODE',
  'source': 'Regex'}]

### SPACY

In [123]:
import spacy
from spacy.tokens import Span

class FlairMatcher(object):
    name = "flair"

    def __init__(self, nlp, language):
        self.tagger = SequenceTagger.load('ner-ontonotes-fast')

    def __call__(self, doc):
        matches = self.tagger.predict(Sentence(doc.text))
        for match in matches[0].get_spans('ner'):
            _match = match.to_dict()
            span = doc.char_span(_match.get('start_pos'), _match.get('end_pos'), label=_match.get('type'))
            doc.ents = list(doc.ents) + [span]
        return doc

In [124]:
nlp = he.load_spacy_model('en', disable=['ner','parser','tagger'])
flair_matcher = FlairMatcher(nlp, 'en')
nlp.add_pipe(flair_matcher)
print(nlp.pipe_names)  # The components in the pipeline

2020-01-10 12:59:19,735 loading file C:\Users\makayser\.flair\models\en-ner-ontonotes-fast-v0.4.pt
['flair']


In [125]:
%%time
doc = nlp("This is a text about Barack Obama and a tree kangaroo. I use Windows 7 at work, ")
print([(ent.text, ent.label_) for ent in doc.ents])

PERSON-span [6,7]: "Barack Obama"
{'text': 'Barack Obama', 'start_pos': 21, 'end_pos': 33, 'type': 'PERSON', 'confidence': 0.8200025260448456}
PRODUCT-span [14,15]: "Windows 7"
{'text': 'Windows 7', 'start_pos': 61, 'end_pos': 70, 'type': 'PRODUCT', 'confidence': 0.7047008872032166}
[('Barack Obama', 'PERSON'), ('Windows 7', 'PRODUCT')]
Wall time: 455 ms


### FLAIR

Good:
- Better quality

Bad:
- Slow

In [2]:
from flair.data import Sentence
from flair.models import SequenceTagger

I0112 00:12:31.152729 32684 file_utils.py:35] PyTorch version 1.3.1 available.


In [5]:
en_tagger = None#SequenceTagger.load('C:/Users/makayser/.flair/models/en-ner-conll03-v0.4.pt')

In [4]:

en = Sentence("My Windows 7 PC has a blank screen after i re-installed it. Not even getting an error code. Can I have Bill Gates number, need help?! Microsoft should know better...")
# load the NER tagger
# en_tagger = SequenceTagger.load('ner-ontonotes-fast')

# run NER over sentence
en_tagger.predict(en)

print(en)
print('The following NER tags are found:')

# iterate over entities and print
for entity in en.get_spans('ner'):
    print(entity)

Sentence: "My Windows 7 PC has a blank screen after i re-installed it. Not even getting an error code. Can I have Bill Gates number, need help?! Microsoft should know better..." - 30 Tokens
The following NER tags are found:
MISC-span [2,3]: "Windows 7"
PER-span [22,23]: "Bill Gates"
ORG-span [27]: "Microsoft"


In [91]:
str(en.get_spans('ner')[0]).split('[')[1].split(']')[0][-1]

'4'

In [29]:
# load the NER tagger
multi_tagger = SequenceTagger.load('ner-multi-fast')

2020-01-09 20:21:01,752 loading file C:\Users\makayser\.flair\models\ner-multi-fast.pt


In [30]:
%%time
# multi = Sentence("My Windows 7 PC has a blank screen after i re-installed it. Not even getting an error code. Can I have Bill Gates number, need help?! Microsoft should know better...")
multi = Sentence("Mein Windows Vista rechner will nicht mehr - ich kriege dauernd fehler meldungen. Was ist Microsoft für eine Firma. Ich wollte mir eh einen neuen kaufen, aber ich hab kein Geld. Kann Bill Gates mir helfen?")

# run NER over sentence
multi_tagger.predict(multi)

print(multi)
print('The following NER tags are found:')

# iterate over entities and print
for entity in en.get_spans('ner'):
    print(entity)

Sentence: "Mein Windows Vista rechner will nicht mehr - ich kriege dauernd fehler meldungen. Was ist Microsoft für eine Firma. Ich wollte mir eh einen neuen kaufen, aber ich hab kein Geld. Kann Bill Gates mir helfen?" - 36 Tokens
The following NER tags are found:
PRODUCT-span [2,3,4]: "Windows 7 PC"
PERSON-span [22,23]: "Bill Gates"
ORG-span [27]: "Microsoft"
Wall time: 821 ms


In [5]:
en = Sentence("My Windows 7 PC has a blank screen after i re-installed it. Not even getting an error code. Can I have Bill Gates number, need help?! Microsoft should know better...")
# load the NER tagger
en_tagger = SequenceTagger.load('ner')

# run NER over sentence
en_tagger.predict(en)

print(en)
print('The following NER tags are found:')

# iterate over entities and print
for entity in en.get_spans('ner'):
    print(entity)

2020-01-09 12:44:20,136 loading file C:\Users\makayser\.flair\models\en-ner-conll03-v0.4.pt
Sentence: "My Windows 7 PC has a blank screen after i re-installed it. Not even getting an error code. Can I have Bill Gates number, need help?! Microsoft should know better..." - 30 Tokens
The following NER tags are found:
MISC-span [2,3]: "Windows 7"
PER-span [22,23]: "Bill Gates"
ORG-span [27]: "Microsoft"


In [36]:
import spacy
nlp = spacy.blank('en_core_web_sm')

ImportError: [E048] Can't import language en_core_web_sm from spacy.lang: No module named 'spacy.lang.en_core_web_sm'

In [7]:
de = Sentence("Mein Windows Vista rechner will nicht mehr - ich kriege dauernd fehler meldungen. Was ist Microsoft für eine Firma. Ich wollte mir eh einen neuen kaufen, aber ich hab kein Geld. Kann Bill Gates mir helfen?")
# load the NER tagger
de_tagger = SequenceTagger.load('de-ner')

# run NER over sentence


2020-01-09 12:47:23,317 loading file C:\Users\makayser\.flair\models\de-ner-conll03-v0.4.pt
Sentence: "Mein Windows Vista rechner will nicht mehr - ich kriege dauernd fehler meldungen. Was ist Microsoft für eine Firma. Ich wollte mir eh einen neuen kaufen, aber ich hab kein Geld. Kann Bill Gates mir helfen?" - 36 Tokens
The following NER tags are found:
ORG-span [16]: "Microsoft"
PER-span [33,34]: "Bill Gates"


In [34]:
%%time
t = " Mein Windows 7 rechner will nicht mehr - ich kriege dauernd fehler meldungen. Was ist Microsoft für eine Firma. Ich wollte mir eh einen neuen kaufen, aber ich hab kein Geld. Kann Bill Gates mir helfen?"
de = Sentence(t + t + t + t )
multi_tagger.predict(de)

print(de)
print('The following NER tags are found:')

# iterate over entities and print
for entity in de.get_spans('ner'):
    print(entity)

Sentence: "Mein Windows 7 rechner will nicht mehr - ich kriege dauernd fehler meldungen. Was ist Microsoft für eine Firma. Ich wollte mir eh einen neuen kaufen, aber ich hab kein Geld. Kann Bill Gates mir helfen? Mein Windows 7 rechner will nicht mehr - ich kriege dauernd fehler meldungen. Was ist Microsoft für eine Firma. Ich wollte mir eh einen neuen kaufen, aber ich hab kein Geld. Kann Bill Gates mir helfen? Mein Windows 7 rechner will nicht mehr - ich kriege dauernd fehler meldungen. Was ist Microsoft für eine Firma. Ich wollte mir eh einen neuen kaufen, aber ich hab kein Geld. Kann Bill Gates mir helfen? Mein Windows 7 rechner will nicht mehr - ich kriege dauernd fehler meldungen. Was ist Microsoft für eine Firma. Ich wollte mir eh einen neuen kaufen, aber ich hab kein Geld. Kann Bill Gates mir helfen?" - 144 Tokens
The following NER tags are found:
MISC-span [2,3]: "Windows 7"
ORG-span [16]: "Microsoft"
PER-span [33,34]: "Bill Gates"
ORG-span [52]: "Microsoft"
PER-span [69,70]: "